In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd
from tensorflow.keras.applications import ResNet50V2

tf.__version__

'2.3.0-dev20200522'

In [3]:
PATH = 'data'

os.listdir(PATH)

['jpeg224',
 'sample_submission.csv',
 'subset.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv']

In [4]:
train = pd.read_csv(f'{PATH}/subset.csv')
val = pd.read_csv(f'{PATH}/val_split.csv')

train.shape, val.shape

((33126, 8), (10932, 8))

In [5]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [6]:
BATCH_SIZE = 64

def decode(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img, label

def load_ds(df):
    imgs, labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/jpeg224/train/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    ds = ds.map(decode)
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [6]:
train_ds = load_ds(train)
val_ds = load_ds(val)

In [7]:
IMAGE_SIZE = (224, 224, 3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)
encoder.trainable = False

inputs = keras.Input(shape=IMAGE_SIZE)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [9]:
model.fit(train_ds, epochs=10, validation_data=val_ds, validation_steps=10)

Epoch 1/10
35/35 [==============================] - 14s 407ms/step - loss: 0.1706 - auc: 0.4480 - val_loss: 0.1717 - val_auc: 0.4202
Epoch 2/10
35/35 [==============================] - 12s 338ms/step - loss: 0.0985 - auc: 0.5623 - val_loss: 0.0821 - val_auc: 0.5170
Epoch 3/10
35/35 [==============================] - 12s 340ms/step - loss: 0.0849 - auc: 0.6974 - val_loss: 0.0770 - val_auc: 0.6066
Epoch 4/10
35/35 [==============================] - 12s 339ms/step - loss: 0.0763 - auc: 0.7708 - val_loss: 0.1082 - val_auc: 0.7238
Epoch 5/10
35/35 [==============================] - 12s 344ms/step - loss: 0.0703 - auc: 0.8309 - val_loss: 0.0696 - val_auc: 0.7243
Epoch 6/10
35/35 [==============================] - 12s 337ms/step - loss: 0.0657 - auc: 0.8745 - val_loss: 0.0785 - val_auc: 0.6919
Epoch 7/10
35/35 [==============================] - 12s 335ms/step - loss: 0.0624 - auc: 0.8961 - val_loss: 0.0669 - val_auc: 0.6677
Epoch 8/10
35/35 [==============================] - 12s 343ms/step - 

In [10]:
test = pd.read_csv(f'{PATH}/test.csv')

test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [11]:
def decode_test(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img

def load_test_ds(df):
    imgs = df["image_name"].values
    imgs = [f'{PATH}/jpeg224/test/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [12]:
test_ds = load_test_ds(test)

In [13]:
preds = model.predict(test_ds)

In [14]:
preds

array([[0.00312989],
       [0.0053902 ],
       [0.00849308],
       ...,
       [0.01873386],
       [0.0010164 ],
       [0.0765562 ]], dtype=float32)

In [15]:
submission = pd.DataFrame({'image_name': test['image_name'].values, 'target': preds.ravel()})

submission

,image_name,target
0,ISIC_0052060,0.003130
1,ISIC_0052349,0.005390
2,ISIC_0058510,0.008493
3,ISIC_0073313,0.000586
4,ISIC_0073502,0.008747
...,...,...
10977,ISIC_9992485,0.015148
10978,ISIC_9996992,0.082123
10979,ISIC_9997917,0.018734
10980,ISIC_9998234,0.001016


In [16]:
submission.to_csv('submission.csv', index=False)